In [7]:
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import mlflow
from mlflow.models.signature import infer_signature
from refactoring import data_loading, preprocess, mlflow_helper, evaluation
# Start the MLflow server
mlflow_helper.start_mlflow()
# Setup MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("wikipedia")

Docker Desktop is already running.
MLflow is already running.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1729469033000, experiment_id='1', last_update_time=1729469033000, lifecycle_stage='active', name='wikipedia', tags={}>

In [11]:
# Setup training data
#columns = ['E_NEds', 'E_Bpag', 'NEds', 'NDays', 'NActDays',
           #'NPages', 'NPcreated', 'pagesWomen', 'wikiprojWomen', 'weightIJ', 'NIJ']
columns = ["E_NEds", "E_Bpag", "firstDay", "lastDay", "NEds", "NDays", "NActDays", "NPages", "NPcreated", "pagesWomen", "wikiprojWomen", "ns_user", "ns_wikipedia", "ns_talk", "ns_userTalk", "ns_content", "weightIJ", "NIJ"]
X_train, X_test, y_train, y_test = preprocess.get_data(columns)


In [6]:
# Random Forest model
mlflow.sklearn.autolog()
with mlflow.start_run() as run:
    rf_model = Pipeline(steps=[
        ('random_forest', RandomForestClassifier(bootstrap = True, max_depth = None, max_features = 'sqrt', min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200, random_state=42))
    ])
    rf_model.fit(X_train, y_train)
    accuracy = rf_model.score(X_test, y_test)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model to MLflow
    mlflow.sklearn.log_model(
        sk_model=rf_model,
        artifact_path="model",
        signature=infer_signature(X_train, rf_model.predict(X_train)),
        input_example= X_train.head(1))


2024/10/20 17:43:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run magnificent-cow-979 at: http://localhost:5000/#/experiments/1/runs/2c494bdaf2fa4fa3819a5ac846f79423.
2024/10/20 17:43:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


MlflowException: API request to http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/1/2c494bdaf2fa4fa3819a5ac846f79423/artifacts/model/conda.yaml failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/1/2c494bdaf2fa4fa3819a5ac846f79423/artifacts/model/conda.yaml (Caused by ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)))

In [4]:

loaded_model,last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")
evaluation.evaluate_model(loaded_model, X_test, y_test, last_run)

Latest run ID: f889c8df9fa543a9b45993ab45e0f598


Error Cuadrático Medio (MSE) - RandomForestClassifier: 0.7389
Accuracy - RandomForestClassifier: 0.557
Informe de clasificación - RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.47      0.42      0.45       307
           1       0.64      0.68      0.66       528
           2       0.22      0.24      0.23        68

    accuracy                           0.56       903
   macro avg       0.45      0.44      0.44       903
weighted avg       0.55      0.56      0.55       903



In [3]:
# Logistic Regression model
with mlflow.start_run() as run:
    logistic_model = Pipeline(steps=[
        ('logistic_regression', LogisticRegression(random_state=42,solver='lbfgs',multi_class='multinomial'))
    ])

    logistic_model.fit(X_train, y_train)
    logistic_accuracy = logistic_model.score(X_test, y_test)
    mlflow.log_metric("logistic_accuracy", logistic_accuracy)

    # Log the model to MLflow
    mlflow.sklearn.log_model(
        sk_model=logistic_model,
        artifact_path="model",
        signature=infer_signature(X_train, rf_model.predict(X_train)),
        input_example= X_train.head(1))

2024/10/20 17:04:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run adorable-asp-649 at: http://localhost:5000/#/experiments/1/runs/31a9903def5a4651b31d9cbda11286a8.
2024/10/20 17:04:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


NameError: name 'X_train' is not defined

In [6]:
loaded_model, last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")
evaluation.evaluate_model(loaded_model, X_test, y_test, last_run)

Latest run ID: 2b441c6cf87b4ab8bc3895f282422365


Error Cuadrático Medio (MSE) - LogisticRegression: 0.918
Accuracy - LogisticRegression: 0.4463
Informe de clasificación - LogisticRegression:
              precision    recall  f1-score   support

           0       0.46      0.46      0.46       307
           1       0.67      0.42      0.52       528
           2       0.14      0.56      0.23        68

    accuracy                           0.45       903
   macro avg       0.43      0.48      0.40       903
weighted avg       0.56      0.45      0.48       903



In [19]:
# Deep Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
import keras
import mlflow.keras
import numpy as np

mlflow.keras.autolog()

with mlflow.start_run() as run:
    input_dim = X_train.shape[1]
    num_classes = len(np.unique(y_train))

    keras.utils.set_random_seed(42)
    model = Sequential()
    model.add(Input(shape=(input_dim,))) 
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    loss = 'sparse_categorical_crossentropy'
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=64,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    mlflow.log_metric("accuracy", accuracy)

    mlflow.keras.log_model(
        model=model,
        artifact_path="model",
    )

2024/10/20 02:31:56 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'pandas.core.frame.DataFrame'>. Dataset logging skipped.
2024/10/20 02:31:56 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'DataFrame' object has no attribute 'flatten'


Epoch 1/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4187 - loss: 1.0734 - val_accuracy: 0.3821 - val_loss: 1.0684
Epoch 2/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4992 - loss: 1.0163 - val_accuracy: 0.4474 - val_loss: 1.0333
Epoch 3/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4990 - loss: 1.0062 - val_accuracy: 0.4396 - val_loss: 1.0292
Epoch 4/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5179 - loss: 0.9900 - val_accuracy: 0.4540 - val_loss: 1.0179
Epoch 5/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5114 - loss: 0.9829 - val_accuracy: 0.4585 - val_loss: 1.0088
Epoch 6/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5196 - loss: 0.9757 - val_accuracy: 0.4673 - val_loss: 1.0087
Epoch 7/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5356 - loss: 0.9633 - val_accuracy: 0.4563 - val_loss: 1.0120
Epoch 8/50
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5305 - loss: 0.9574 - val_accuracy: 0.4385 - val_loss:

2024/10/20 02:32:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/20 02:32:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 02:32:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-horse-353 at: http://localhost:5000/#/experiments/3/runs/dff792fed79140979cc431ab48f163ef.
2024/10/20 02:32:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

loaded_model, last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")

# Get predicted probabilities
y_pred_proba = loaded_model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_proba, axis=1)

# Compute classification metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print metrics
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

Latest run ID: dff792fed79140979cc431ab48f163ef


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.4673311184939092
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.54      0.49       307
           1       0.70      0.42      0.52       528
           2       0.16      0.51      0.24        68

    accuracy                           0.47       903
   macro avg       0.44      0.49      0.42       903
weighted avg       0.57      0.47      0.49       903



In [9]:
# API TEST
# docker build -t fastapi-mlflow-app .
# docker run -p 8000:8000 fastapi-mlflow-app
import requests
import json

# Define the API endpoint
url = "http://localhost:8000/train"

# Define the data payload
payload = {
    "X_train": [[1.0, 2.0], [3.0, 4.0]],
    "y_train": [0, 1],
    "X_test": [[5.0, 6.0]],
    "y_test": [1],
    "classifier_type": "logistic_regression",
    "hyperparameters": {"C": 1.0, "max_iter": 100}
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, data=json.dumps(payload), headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    print(f"Response: {response.json()}")
else:
    print(f"Request failed with status code {response.status_code}")
    print(f"Error: {response.text}")

Request was successful!
Response: {'status': 'Model trained and logged successfully'}


In [12]:
loaded_model, last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")
evaluation.evaluate_model(loaded_model, X_test, y_test, last_run)

Latest run ID: 2e9f29d8e20a4588ab25163618de1b17


2024/10/20 20:27:20 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.16.2, required: mlflow==2.17.0)
 - cloudpickle (current: 3.0.0, required: cloudpickle==3.1.0)
 - numpy (current: 1.26.4, required: numpy==2.1.2)
 - pandas (current: 2.2.2, required: pandas==2.2.3)
 - scikit-learn (current: 1.5.0, required: scikit-learn==1.5.2)
 - scipy (current: 1.13.1, required: scipy==1.14.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/10/20 20:27:20 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.15`, differs from the version of Python that is currently running, `Python 3.12.6`, and may be incompatible


Error Cuadrático Medio (MSE) - Pipeline: 0.918
Accuracy - Pipeline: 0.4463
Informe de clasificación - Pipeline:
              precision    recall  f1-score   support

           0       0.46      0.46      0.46       307
           1       0.67      0.42      0.52       528
           2       0.14      0.56      0.23        68

    accuracy                           0.45       903
   macro avg       0.43      0.48      0.40       903
weighted avg       0.56      0.45      0.48       903



c:\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
